# Modelo de Suavización Exponencial - Promedio Móvil Simple (SMA)

Este notebook implementa el método de Promedio Móvil Simple (SMA) para predecir las ventas de dos productos, utilizando tres tipos de validación temporal:
- Walk-Forward Validation
- Rolling Window
- Expanding Window


In [31]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')


In [32]:
# Cargar el dataset
df = pd.read_csv('data-set.csv', index_col=0)
print("Dataset cargado:")
print(f"Forma del dataset: {df.shape}")
print(f"\nPrimeras filas:")
print(df.head())
print(f"\nÚltimas filas:")
print(df.tail())
print(f"\nInformación del dataset:")
print(df.info())


Dataset cargado:
Forma del dataset: (127, 2)

Primeras filas:
    producto1   producto2
1  500.000000  200.000000
2  497.400893  210.686220
3  478.605317  222.018584
4  486.454125  233.920990
5  479.695678  238.402098

Últimas filas:
      producto1   producto2
123  164.610771  629.293034
124  150.881839  637.099467
125  151.788470  653.155282
126  137.047639  672.528345
127  141.990873  676.058092

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 1 to 127
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   producto1  127 non-null    float64
 1   producto2  127 non-null    float64
dtypes: float64(2)
memory usage: 3.0 KB
None


## Función SMA (Simple Moving Average)  

Implementación del método de Promedio Móvil Simple para realizar predicciones.


In [33]:
def sma_forecast(data, window_size, forecast_horizon=1):
    """
    Calcula el promedio móvil simple y realiza predicciones.
    
    Parameters:
    -----------
    data : array-like
        Serie temporal de datos
    window_size : int
        Tamaño de la ventana para el promedio móvil
    forecast_horizon : int
        Número de períodos a predecir (por defecto 1)
    
    Returns:
    --------
    float
        Predicción usando SMA
    """
    if len(data) < window_size:
        # Si no hay suficientes datos, usar el promedio de todos los disponibles
        return np.mean(data)
    
    # Calcular el promedio móvil simple de los últimos 'window_size' valores
    sma_value = np.mean(data[-window_size:])
    
    # Para SMA, todas las predicciones futuras son iguales al último SMA calculado
    return sma_value


## Función 1: Walk-Forward Validation
    
En Walk-Forward Validation, se entrena el modelo con datos hasta un punto específico y se valida con el siguiente punto. Luego se avanza un paso y se repite el proceso.


In [34]:
def walk_forward_validation(dataset, window_size=5, train_size_min=10):
    """
    Realiza validación Walk-Forward para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    window_size : int
        Tamaño de la ventana para SMA (por defecto 5)
    train_size_min : int
        Tamaño mínimo de entrenamiento antes de comenzar la validación
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Walk-Forward Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            prediction = sma_forecast(train_data, window_size, forecast_horizon=1)
            
            # Calcular métricas
            error = actual - prediction
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana_train': i,
                'indice_test': i,
                'actual': actual,
                'prediccion': prediction,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 2: Rolling Window Validation

En Rolling Window Validation, se mantiene un tamaño fijo de ventana de entrenamiento que se desplaza a lo largo del tiempo.


In [35]:
def rolling_window_validation(dataset, window_size=5, train_window_size=20):
    """
    Realiza validación Rolling Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    window_size : int
        Tamaño de la ventana para SMA (por defecto 5)
    train_window_size : int
        Tamaño fijo de la ventana de entrenamiento
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Rolling Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar desde train_window_size hasta n-1
        for i in range(train_window_size, n):
            # Datos de entrenamiento: ventana fija de tamaño train_window_size
            train_data = data[i - train_window_size:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            prediction = sma_forecast(train_data, window_size, forecast_horizon=1)
            
            # Calcular métricas
            error = actual - prediction
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_window_size + 1,
                'ventana_train_inicio': i - train_window_size,
                'ventana_train_fin': i,
                'indice_test': i,
                'actual': actual,
                'prediccion': prediction,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_window_size + 1) % 20 == 0:
                print(f"  Iteración {i - train_window_size + 1}/{n - train_window_size}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 3: Expanding Window Validation

En Expanding Window Validation, la ventana de entrenamiento crece con cada iteración, comenzando desde un tamaño mínimo y expandiéndose hasta incluir todos los datos disponibles hasta ese punto.


In [36]:
def expanding_window_validation(dataset, window_size=5, train_size_min=10):
    """
    Realiza validación Expanding Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    window_size : int
        Tamaño de la ventana para SMA (por defecto 5)
    train_size_min : int
        Tamaño mínimo inicial de la ventana de entrenamiento
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Expanding Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i (ventana que se expande)
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            prediction = sma_forecast(train_data, window_size, forecast_horizon=1)
            
            # Calcular métricas
            error = actual - prediction
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana_train': i,  # Tamaño de la ventana que se expande
                'indice_test': i,
                'actual': actual,
                'prediccion': prediction,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Ejecución de las Validaciones

Después de definir las funciones que permitirán ejecutar el modelo teniendo en cuenta los tres tipos de validacion, ahora ejecutamos las tres validaciones para cada producto y calculamos las métricas promedio.


In [37]:
# Parámetros de validación
WINDOW_SIZE = 5  # Tamaño de ventana para SMA
TRAIN_SIZE_MIN = 10  # Tamaño mínimo de entrenamiento
TRAIN_WINDOW_SIZE = 20  # Tamaño fijo para Rolling Window

# Ejecutar Walk-Forward Validation
print("=" * 60)
print("WALK-FORWARD VALIDATION")
print("=" * 60)
results_wf = walk_forward_validation(df, window_size=WINDOW_SIZE, train_size_min=TRAIN_SIZE_MIN)


WALK-FORWARD VALIDATION

=== Walk-Forward Validation para producto1 ===
  Iteración 20/117: RMSE=9.7747, MAE=9.7747
  Iteración 40/117: RMSE=19.9284, MAE=19.9284
  Iteración 60/117: RMSE=13.1798, MAE=13.1798
  Iteración 80/117: RMSE=22.8071, MAE=22.8071
  Iteración 100/117: RMSE=27.4172, MAE=27.4172
  Total de iteraciones: 117

=== Walk-Forward Validation para producto2 ===
  Iteración 20/117: RMSE=37.1870, MAE=37.1870
  Iteración 40/117: RMSE=38.2643, MAE=38.2643
  Iteración 60/117: RMSE=20.6616, MAE=20.6616
  Iteración 80/117: RMSE=6.1219, MAE=6.1219
  Iteración 100/117: RMSE=53.5314, MAE=53.5314
  Total de iteraciones: 117


In [38]:
# Ejecutar Rolling Window Validation
print("\n" + "=" * 60)
print("ROLLING WINDOW VALIDATION")
print("=" * 60)
results_rw = rolling_window_validation(df, window_size=WINDOW_SIZE, train_window_size=TRAIN_WINDOW_SIZE)



ROLLING WINDOW VALIDATION

=== Rolling Window Validation para producto1 ===
  Iteración 20/107: RMSE=19.4222, MAE=19.4222
  Iteración 40/107: RMSE=19.2855, MAE=19.2855
  Iteración 60/107: RMSE=2.2361, MAE=2.2361
  Iteración 80/107: RMSE=18.5612, MAE=18.5612
  Iteración 100/107: RMSE=14.4655, MAE=14.4655
  Total de iteraciones: 107

=== Rolling Window Validation para producto2 ===
  Iteración 20/107: RMSE=8.7254, MAE=8.7254
  Iteración 40/107: RMSE=31.3087, MAE=31.3087
  Iteración 60/107: RMSE=14.6484, MAE=14.6484
  Iteración 80/107: RMSE=4.0050, MAE=4.0050
  Iteración 100/107: RMSE=10.0434, MAE=10.0434
  Total de iteraciones: 107


In [39]:
# Ejecutar Expanding Window Validation
print("\n" + "=" * 60)
print("EXPANDING WINDOW VALIDATION")
print("=" * 60)
results_ew = expanding_window_validation(df, window_size=WINDOW_SIZE, train_size_min=TRAIN_SIZE_MIN)



EXPANDING WINDOW VALIDATION

=== Expanding Window Validation para producto1 ===
  Iteración 20/117: RMSE=9.7747, MAE=9.7747
  Iteración 40/117: RMSE=19.9284, MAE=19.9284
  Iteración 60/117: RMSE=13.1798, MAE=13.1798
  Iteración 80/117: RMSE=22.8071, MAE=22.8071
  Iteración 100/117: RMSE=27.4172, MAE=27.4172
  Total de iteraciones: 117

=== Expanding Window Validation para producto2 ===
  Iteración 20/117: RMSE=37.1870, MAE=37.1870
  Iteración 40/117: RMSE=38.2643, MAE=38.2643
  Iteración 60/117: RMSE=20.6616, MAE=20.6616
  Iteración 80/117: RMSE=6.1219, MAE=6.1219
  Iteración 100/117: RMSE=53.5314, MAE=53.5314
  Total de iteraciones: 117


In [40]:
def calcular_metricas_promedio(results, nombre_validacion):
    """
    Calcula las métricas promedio para cada producto.
    
    Parameters:
    -----------
    results : dict
        Resultados de la validación
    nombre_validacion : str
        Nombre del tipo de validación
    
    Returns:
    --------
    dict
        Diccionario con métricas promedio por producto
    """
    metricas_promedio = {}
    
    print(f"\n{'='*60}")
    print(f"MÉTRICAS PROMEDIO - {nombre_validacion.upper()}")
    print(f"{'='*60}")
    
    for producto in results.keys():
        metrics_list = results[producto]
        
        # Calcular promedios
        rmse_promedio = np.mean([m['rmse'] for m in metrics_list])
        mae_promedio = np.mean([m['mae'] for m in metrics_list])
        
        # Calcular RMSE global (sobre todas las predicciones)
        actuales = [m['actual'] for m in metrics_list]
        predicciones = [m['prediccion'] for m in metrics_list]
        rmse_global = np.sqrt(mean_squared_error(actuales, predicciones))
        mae_global = mean_absolute_error(actuales, predicciones)
        
        metricas_promedio[producto] = {
            'rmse_promedio': rmse_promedio,
            'mae_promedio': mae_promedio,
            'rmse_global': rmse_global,
            'mae_global': mae_global,
            'num_iteraciones': len(metrics_list)
        }
        
        print(f"\n{producto}:")
        print(f"  RMSE Promedio (por iteración): {rmse_promedio:.4f}")
        print(f"  MAE Promedio (por iteración): {mae_promedio:.4f}")
        print(f"  RMSE Global (sobre todas las predicciones): {rmse_global:.4f}")
        print(f"  MAE Global (sobre todas las predicciones): {mae_global:.4f}")
        print(f"  Número de iteraciones: {len(metrics_list)}")
    
    return metricas_promedio

# Calcular métricas promedio para cada tipo de validación
metricas_wf = calcular_metricas_promedio(results_wf, "Walk-Forward")
metricas_rw = calcular_metricas_promedio(results_rw, "Rolling Window")
metricas_ew = calcular_metricas_promedio(results_ew, "Expanding Window")



MÉTRICAS PROMEDIO - WALK-FORWARD

producto1:
  RMSE Promedio (por iteración): 17.0308
  MAE Promedio (por iteración): 17.0308
  RMSE Global (sobre todas las predicciones): 20.3711
  MAE Global (sobre todas las predicciones): 17.0308
  Número de iteraciones: 117

producto2:
  RMSE Promedio (por iteración): 23.3543
  MAE Promedio (por iteración): 23.3543
  RMSE Global (sobre todas las predicciones): 27.5841
  MAE Global (sobre todas las predicciones): 23.3543
  Número de iteraciones: 117

MÉTRICAS PROMEDIO - ROLLING WINDOW

producto1:
  RMSE Promedio (por iteración): 17.9499
  MAE Promedio (por iteración): 17.9499
  RMSE Global (sobre todas las predicciones): 21.1199
  MAE Global (sobre todas las predicciones): 17.9499
  Número de iteraciones: 107

producto2:
  RMSE Promedio (por iteración): 23.1984
  MAE Promedio (por iteración): 23.1984
  RMSE Global (sobre todas las predicciones): 27.6627
  MAE Global (sobre todas las predicciones): 23.1984
  Número de iteraciones: 107

MÉTRICAS PROM

## Comparación de Resultados y Selección del Mejor Modelo

Comparamos los RMSE promedio de cada tipo de validación para seleccionar el mejor método.


In [41]:
# Crear DataFrame comparativo
comparacion = []

for producto in df.columns:
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Walk-Forward',
        'RMSE_Promedio': metricas_wf[producto]['rmse_promedio'],
        'RMSE_Global': metricas_wf[producto]['rmse_global'],
        'MAE_Promedio': metricas_wf[producto]['mae_promedio'],
        'MAE_Global': metricas_wf[producto]['mae_global']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Rolling Window',
        'RMSE_Promedio': metricas_rw[producto]['rmse_promedio'],
        'RMSE_Global': metricas_rw[producto]['rmse_global'],
        'MAE_Promedio': metricas_rw[producto]['mae_promedio'],
        'MAE_Global': metricas_rw[producto]['mae_global']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Expanding Window',
        'RMSE_Promedio': metricas_ew[producto]['rmse_promedio'],
        'RMSE_Global': metricas_ew[producto]['rmse_global'],
        'MAE_Promedio': metricas_ew[producto]['mae_promedio'],
        'MAE_Global': metricas_ew[producto]['mae_global']
    })

df_comparacion = pd.DataFrame(comparacion)
print("\n" + "="*80)
print("COMPARACIÓN DE RESULTADOS")
print("="*80)
print(df_comparacion.to_string(index=False))



COMPARACIÓN DE RESULTADOS
 Producto       Validacion  RMSE_Promedio  RMSE_Global  MAE_Promedio  MAE_Global
producto1     Walk-Forward      17.030838    20.371111     17.030838   17.030838
producto1   Rolling Window      17.949853    21.119902     17.949853   17.949853
producto1 Expanding Window      17.030838    20.371111     17.030838   17.030838
producto2     Walk-Forward      23.354261    27.584135     23.354261   23.354261
producto2   Rolling Window      23.198448    27.662746     23.198448   23.198448
producto2 Expanding Window      23.354261    27.584135     23.354261   23.354261


In [42]:
# Encontrar el mejor RMSE para cada producto
print("\n" + "="*80)
print("MEJOR MÉTODO POR PRODUCTO (basado en RMSE Global)")
print("="*80)

mejores_metodos = {}

for producto in df.columns:
    # Comparar RMSE global de cada método
    rmse_wf = metricas_wf[producto]['rmse_global']
    rmse_rw = metricas_rw[producto]['rmse_global']
    rmse_ew = metricas_ew[producto]['rmse_global']
    
    metodos = {
        'Walk-Forward': rmse_wf,
        'Rolling Window': rmse_rw,
        'Expanding Window': rmse_ew
    }
    
    mejor_metodo = min(metodos, key=metodos.get)
    mejor_rmse = metodos[mejor_metodo]
    
    mejores_metodos[producto] = {
        'metodo': mejor_metodo,
        'rmse': mejor_rmse,
        'mae': metricas_wf[producto]['mae_global'] if mejor_metodo == 'Walk-Forward' else \
               metricas_rw[producto]['mae_global'] if mejor_metodo == 'Rolling Window' else \
               metricas_ew[producto]['mae_global']
    }
    
    print(f"\n{producto}:")
    print(f"  Mejor método: {mejor_metodo}")
    print(f"  RMSE Global: {mejor_rmse:.4f}")
    print(f"  MAE Global: {mejores_metodos[producto]['mae']:.4f}")
    print(f"\n  Comparación:")
    print(f"    Walk-Forward:     RMSE = {rmse_wf:.4f}")
    print(f"    Rolling Window:  RMSE = {rmse_rw:.4f}")
    print(f"    Expanding Window: RMSE = {rmse_ew:.4f}")

print("\n" + "="*80)



MEJOR MÉTODO POR PRODUCTO (basado en RMSE Global)

producto1:
  Mejor método: Walk-Forward
  RMSE Global: 20.3711
  MAE Global: 17.0308

  Comparación:
    Walk-Forward:     RMSE = 20.3711
    Rolling Window:  RMSE = 21.1199
    Expanding Window: RMSE = 20.3711

producto2:
  Mejor método: Walk-Forward
  RMSE Global: 27.5841
  MAE Global: 23.3543

  Comparación:
    Walk-Forward:     RMSE = 27.5841
    Rolling Window:  RMSE = 27.6627
    Expanding Window: RMSE = 27.5841



## Modelo Final para Producción

Entrenamos el modelo final usando todos los datos disponibles y el mejor método de validación para realizar predicciones de producción.


In [43]:
def modelo_final_produccion(dataset, window_size=5, horizonte_prediccion=1):
    """
    Entrena el modelo final usando todos los datos disponibles y realiza predicciones.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset completo con todos los datos históricos
    window_size : int
        Tamaño de la ventana para SMA
    horizonte_prediccion : int
        Número de períodos futuros a predecir
    
    Returns:
    --------
    dict
        Diccionario con predicciones para cada producto
    """
    predicciones = {}
    
    print("="*80)
    print("MODELO FINAL PARA PRODUCCIÓN")
    print("="*80)
    
    for producto in dataset.columns:
        print(f"\n{producto}:")
        data = dataset[producto].values
        
        # Usar todos los datos disponibles para entrenar
        # Para SMA, simplemente calculamos el promedio de los últimos window_size valores
        sma_final = np.mean(data[-window_size:])
        
        # Realizar predicciones futuras
        # Para SMA, todas las predicciones futuras son iguales al último SMA calculado
        predicciones_futuras = [sma_final] * horizonte_prediccion
        
        predicciones[producto] = {
            'sma_value': sma_final,
            'predicciones': predicciones_futuras,
            'window_size': window_size,
            'ultimos_valores': data[-window_size:].tolist(),
            'ultimo_valor_real': data[-1]
        }
        
        print(f"  Tamaño de ventana SMA: {window_size}")
        print(f"  Últimos {window_size} valores: {data[-window_size:]}")
        print(f"  SMA calculado: {sma_final:.4f}")
        print(f"  Último valor real: {data[-1]:.4f}")
        print(f"  Predicción para el siguiente período: {sma_final:.4f}")
        if horizonte_prediccion > 1:
            print(f"  Predicciones para los próximos {horizonte_prediccion} períodos: {predicciones_futuras}")
    
    return predicciones

# Entrenar modelo final y realizar predicciones
predicciones_finales = modelo_final_produccion(df, window_size=WINDOW_SIZE, horizonte_prediccion=1)


MODELO FINAL PARA PRODUCCIÓN

producto1:
  Tamaño de ventana SMA: 5
  Últimos 5 valores: [164.6107711 150.8818393 151.7884696 137.0476394 141.9908727]
  SMA calculado: 149.2639
  Último valor real: 141.9909
  Predicción para el siguiente período: 149.2639

producto2:
  Tamaño de ventana SMA: 5
  Últimos 5 valores: [629.2930339 637.0994671 653.1552818 672.5283447 676.0580921]
  SMA calculado: 653.6268
  Último valor real: 676.0581
  Predicción para el siguiente período: 653.6268


## Resumen Final

Resumen de los resultados obtenidos con el método SMA.


In [44]:
print("\n" + "="*80)
print("RESUMEN FINAL")
print("="*80)

print("\nParámetros utilizados:")
print(f"  Tamaño de ventana SMA: {WINDOW_SIZE}")
print(f"  Tamaño mínimo de entrenamiento: {TRAIN_SIZE_MIN}")
print(f"  Tamaño de ventana fija (Rolling): {TRAIN_WINDOW_SIZE}")

print("\nMejor método de validación por producto:")
for producto, info in mejores_metodos.items():
    print(f"\n  {producto}:")
    print(f"    Método: {info['metodo']}")
    print(f"    RMSE Global: {info['rmse']:.4f}")
    print(f"    MAE Global: {info['mae']:.4f}")

print("\nPredicciones finales para producción:")
for producto, pred in predicciones_finales.items():
    print(f"\n  {producto}:")
    print(f"    Predicción siguiente período: {pred['predicciones'][0]:.4f}")
    print(f"    Basado en SMA de los últimos {pred['window_size']} valores")



RESUMEN FINAL

Parámetros utilizados:
  Tamaño de ventana SMA: 5
  Tamaño mínimo de entrenamiento: 10
  Tamaño de ventana fija (Rolling): 20

Mejor método de validación por producto:

  producto1:
    Método: Walk-Forward
    RMSE Global: 20.3711
    MAE Global: 17.0308

  producto2:
    Método: Walk-Forward
    RMSE Global: 27.5841
    MAE Global: 23.3543

Predicciones finales para producción:

  producto1:
    Predicción siguiente período: 149.2639
    Basado en SMA de los últimos 5 valores

  producto2:
    Predicción siguiente período: 653.6268
    Basado en SMA de los últimos 5 valores


## Exportación de Métricas

Exportamos las métricas de cada validación para análisis posterior.


In [45]:
# Exportar métricas detalladas a DataFrames
def exportar_metricas(results, nombre_validacion):
    """
    Exporta las métricas a un DataFrame.
    
    Parameters:
    -----------
    results : dict
        Resultados de la validación
    nombre_validacion : str
        Nombre del tipo de validación
    
    Returns:
    --------
    dict
        Diccionario con DataFrames por producto
    """
    dataframes = {}
    
    for producto in results.keys():
        metrics_list = results[producto]
        df_metrics = pd.DataFrame(metrics_list)
        dataframes[producto] = df_metrics
        
        # Guardar en CSV
        filename = f'metricas_{nombre_validacion.replace(" ", "_")}_{producto}.csv'
        df_metrics.to_csv("metricas/"+"modelo-SMA-"+filename, index=False)
        print(f"Métricas de {producto} ({nombre_validacion}) exportadas a: {filename}")
    
    return dataframes

# Exportar todas las métricas
print("\nExportando métricas...")
metricas_wf_df = exportar_metricas(results_wf, "Walk-Forward")
metricas_rw_df = exportar_metricas(results_rw, "Rolling Window")
metricas_ew_df = exportar_metricas(results_ew, "Expanding Window")

print("\n¡Exportación completada!")



Exportando métricas...
Métricas de producto1 (Walk-Forward) exportadas a: metricas_Walk-Forward_producto1.csv
Métricas de producto2 (Walk-Forward) exportadas a: metricas_Walk-Forward_producto2.csv
Métricas de producto1 (Rolling Window) exportadas a: metricas_Rolling_Window_producto1.csv
Métricas de producto2 (Rolling Window) exportadas a: metricas_Rolling_Window_producto2.csv
Métricas de producto1 (Expanding Window) exportadas a: metricas_Expanding_Window_producto1.csv
Métricas de producto2 (Expanding Window) exportadas a: metricas_Expanding_Window_producto2.csv

¡Exportación completada!
